In [1]:
import torch
from settings import *
from lrp_resnet_canonized_poolconv_prototypes import *
from preprocess import *
from PIL import Image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.autograd import Variable
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import model


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/kr21883/Library/CloudStorage/OneDrive-UniversityofBristol/Code/PRP_Label_Imbalance/venv_xrays/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kr21883/Library/CloudStorage/OneDrive-UniversityofBristol/Code/PRP_Label_Im

In [2]:
device = 'cpu'
if torch.cuda.is_available():
    device = "cuda"
#elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
#    device = "mps"
print(f"using device: {device}")

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load trained ProtoPNet model
load_model_path = os.path.join(
    load_model_dir, base_architecture, experiment_run, load_model_name)

ppnet = model.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)

ppnet.load_state_dict(torch.load(
    load_model_path, map_location=device), strict=False)
prp_model = make_prp_model(ppnet, device)


using device: cpu
at src_module_name 


at src_module_name conv1
is Conv2d


at src_module_name bn1
is BatchNorm2d
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
conv2d_zbeta_wrapper_class(
  (module): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
)
zeroparam_wrapper_class(
  (module): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


at src_module_name relu


at src_module_name maxpool


at src_module_name layer1


at src_module_name layer1.0


at src_module_name layer1.0.conv1
is Conv2d


at src_module_name layer1.0.bn1
is BatchNorm2d
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
oneparam_wrapper_class(
  (module): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
zeroparam_wrapper_class(
  (modu

In [3]:
# For a single test image and for a particular prototype
test_image_path = os.path.join(test_image_dir, test_image_name)


In [9]:
torch.asarray(np.array(Image.open(test_image_path)
                             ).transpose([2, 0, 1]))

RuntimeError: could not retrieve buffer from object

In [5]:
torch.asarray(np.array(Image.open(test_image_path)
                             ).transpose([2, 0, 1])).float()

RuntimeError: could not retrieve buffer from object

In [6]:

img = torch.asarray(np.array(Image.open(test_image_path)
                             ).transpose([2, 0, 1])).float()
img_variable = Variable(img.unsqueeze(0))/255
img_tensor = preprocess(img_variable, mean, std)
images_test = img_tensor.to(device)

prp_map = run_prp_image(prp_model, images_test, prototype_number, device)
makedir(write_path)
plt.imsave(write_path+"prp_"+str(prototype_number)+"_" +
           test_image_name, prp_map, cmap="seismic", vmin=-1, vmax=+1)

RuntimeError: could not retrieve buffer from object

In [ ]:

#############
### OVERLAY ##########
################
def invert_normalize(ten, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    print(ten.shape)
    s = torch.tensor(np.asarray(std, dtype=np.float32)
                     ).unsqueeze(1).unsqueeze(2)
    m = torch.tensor(np.asarray(mean, dtype=np.float32)
                     ).unsqueeze(1).unsqueeze(2)

    res = ten * s + m
    return res


heatmap = cv2.imread("Test images/PRP/"+"prp_" +
                     str(prototype_number)+"_"+test_image_name)
heatmap = heatmap[..., ::-1]
heatmap = np.float32(heatmap) / 255
ts = invert_normalize(img_tensor.squeeze())
a = ts.data.numpy().transpose((1, 2, 0))
overlayed_original_img_j = 0.2 * a + 0.6 * heatmap
plt.imsave("Test images/PRP/"+"Overlay_prp_"+str(prototype_number)+"_"+test_image_name,
           overlayed_original_img_j,
           vmin=-1,
           vmax=+1.0)
